<a href="https://colab.research.google.com/github/sakibchowdhury131/railcop/blob/main/Bonsai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/microsoft/EdgeML.git

fatal: destination path 'EdgeML' already exists and is not an empty directory.


In [2]:
%cd /content/EdgeML/tf/

/content/EdgeML/tf


In [3]:
!pwd

/content/EdgeML/tf


In [3]:
!pip install -r requirements-cpu.txt
!pip install -e .

  Using cached numpy-1.16.4-cp37-cp37m-manylinux1_x86_64.whl (17.3 MB)
  Using cached pandas-0.23.4-cp37-cp37m-manylinux1_x86_64.whl (8.8 MB)
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 385, in run
    conflict

In [44]:
%cd /content/EdgeML/examples/tf/Bonsai

/content/EdgeML/examples/tf/Bonsai


In [45]:
import helpermethods

In [4]:
import tensorflow as tf
import numpy as np
import sys
import os

#Provide the GPU number to be used
os.environ['CUDA_VISIBLE_DEVICES'] =''

#Bonsai imports
from edgeml_tf.trainer.bonsaiTrainer import BonsaiTrainer
from edgeml_tf.graph.bonsai import Bonsai

# Fixing seeds for reproducibility
tf.set_random_seed(42)
np.random.seed(42)

In [5]:
import pandas as pd
print(pd.__version__)

1.3.4


In [6]:
label0 = pd.read_csv('/content/drive/MyDrive/RailCop/New Files/Sakib/data/train_7_label0.csv')
label1 = pd.read_csv('/content/drive/MyDrive/RailCop/New Files/Sakib/data/train_7_label1.csv')
print(label0)

      Sensor1  ...  Sensor1+Sensor2+Sensor3+Sensor4
0          21  ...                               31
1          11  ...                               21
2          11  ...                               20
3          12  ...                               22
4          10  ...                               19
...       ...  ...                              ...
5295       10  ...                               20
5296       11  ...                               22
5297       12  ...                               23
5298       10  ...                               19
5299        9  ...                               18

[5300 rows x 15 columns]


In [7]:
__label0 = label0.to_numpy()
__label1 = label1.to_numpy()[0:5300, :]
print(__label0)

[[21  6  1 ... 25 10 31]
 [11  5  2 ... 16 10 21]
 [11  4  3 ... 16  9 20]
 ...
 [12  6  1 ... 17 11 23]
 [10  5  3 ... 14  9 19]
 [ 9  5  3 ... 13  9 18]]


In [8]:
Y0 = np.zeros(__label0.shape[0])
Y1 = np.ones(__label1.shape[0])
print(Y0.shape)
print(Y1.shape)

(5300,)
(5300,)


In [9]:
X = np.asarray(np.concatenate((__label0, __label1), axis = 0), dtype=np.float32)
Y = np.asarray(np.concatenate((Y0, Y1), axis = 0), dtype = np.float32)
print(X.shape)
print(Y.shape)

(10600, 15)
(10600,)


In [10]:
from sklearn.utils import shuffle
X, Y = shuffle(X,Y)

In [23]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.33, random_state=42)

In [15]:
sigma = 1.0 #Sigmoid parameter for tanh
depth = 3 #Depth of Bonsai Tree
projectionDimension = 28 #Lower Dimensional space for Bonsai to work on

#Regularizers for Bonsai Parameters
regZ = 0.0001
regW = 0.001
regV = 0.001
regT = 0.001

totalEpochs = 100

learningRate = 0.01

outFile = None

#Sparsity for Bonsai Parameters. x => 100*x % are non-zeros
sparZ = 0.2
sparW = 0.3
sparV = 0.3
sparT = 0.62

batchSize = np.maximum(100, int(np.ceil(np.sqrt(Ytrain.shape[0]))))

useMCHLoss = True #only for Multiclass cases True: Multiclass-Hing Loss, False: Cross Entropy. 

#Bonsai uses one classier for Binary, thus this condition

numClasses = 1

In [16]:
Xtrain.shape

(7102, 15)

In [46]:
__X = tf.placeholder("float32", [None, Xtrain.shape[1]])
__Y = tf.placeholder("float32", [None, numClasses])

In [47]:
bonsaiObj = Bonsai(numClasses, Xtrain.shape[1], projectionDimension, depth, sigma)

In [48]:
bonsaiTrainer = BonsaiTrainer(bonsaiObj, regW, regT, regV, regZ, sparW, sparT, sparV, sparZ,
                              learningRate, __X, __Y, useMCHLoss, outFile)

In [49]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [55]:
Ytrain.shape

(7102, 1)

In [54]:
Ytrain = np.expand_dims(Ytrain,1)
Ytest = np.expand_dims(Ytest,1)

In [56]:
bonsaiTrainer.train(batchSize, totalEpochs, sess,
                    Xtrain, Xtest, Ytrain, Ytest, '/content', '/content')


Epoch Number: 0

******************** Dense Training Phase Started ********************


Classification Train Loss: 1.096813075978991
Training accuracy (Classification): 0.5823943669527349
Test accuracy 0.514294
MarginLoss + RegLoss: 1.1850945 + 0.18597856 = 1.371073


Epoch Number: 1

Classification Train Loss: 1.163481355552942
Training accuracy (Classification): 0.5609859156776482
Test accuracy 0.499142
MarginLoss + RegLoss: 3.2428083 + 0.17940792 = 3.4222162


Epoch Number: 2

Classification Train Loss: 1.0595734161390384
Training accuracy (Classification): 0.6008450703721651
Test accuracy 0.498856
MarginLoss + RegLoss: 3.4842536 + 0.17420791 = 3.6584616


Epoch Number: 3

Classification Train Loss: 1.0287369095103842
Training accuracy (Classification): 0.6164788763288042
Test accuracy 0.499142
MarginLoss + RegLoss: 3.716603 + 0.16973017 = 3.8863332


Epoch Number: 4

Classification Train Loss: 1.0453356638760634
Training accuracy (Classification): 0.605492952843787
Test accuracy